<a href="https://colab.research.google.com/github/LEVIII007/chess/blob/main/Copy_of_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'chess-games:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1159749%2F1944111%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240127%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240127T165103Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17ff3ee24a50bc94c52d4b441ff14c852ead062910161e09dc5c075a751d42597e75b115dee1a7e3ca5945d735e107d37021cfb22a223c8162f246982e23978d1541b38c6a3aa2b2fd5ebd0b29e10f37896248fd6d0bba28a62f0db97232db005a66a1fcf67b5d6376f0937d7d42a224e934d2ff34ab794db52e1986c7be6820f6f01c90d716e4fa446972e69df9fc39072781e1e24a33519647beed500268bf0440ec26a69bb97bfb85091d67a09370bd98818f9dc014374a3f07963a452bb45cb0a17a10021df9f37d65fe8e0b87289d476acdcd9e437256397fb0ba266d57f92de7eb7e75fe1d168a92192c17cd24a9cfe5ce1515c8001a321d9582480fbb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1555836415 bytes downloaded
Downloaded and uncompressed: chess-games
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install chess
!pip install pytorch

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import re
import numpy as np
import chess
import pandas as pd
import gc
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
letter_2_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_2_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

In [ ]:
import chess

# Create a chess board
board = chess.Board()

# Print the chess board in characters
print(board)

**2.**

In [ ]:
#making chess board layers
def create_rep_layer(board, type):

    s = str(board)
    s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
    s = re.sub(f'{type}', '-1', s)
    s = re.sub(f'{type.upper()}', '1', s)
    s = re.sub(f'\.', '0', s)

    board_mat = []
    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        board_mat.append(row)

    return np.array(board_mat)

**1**

In [ ]:
def board_2_rep(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k']
    layers = []
    for piece in pieces:
        layers.append(create_rep_layer(board, piece))
    board_rep = np.stack(layers)
    return board_rep

In [ ]:
#converting moves in a list form
def create_move_list(s):
    return re.sub('\d*\. ','',s).split(' ')[:-1]

In [ ]:
#single move ki representation
def move_2_rep(move, board):
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8,8))
    from_row = 8 - int(move[1])
    from_column = letter_2_num[move[0]]
    from_output_layer[from_row, from_column] = 1

    to_output_layer = np.zeros((8,8))
    to_row = 8 - int(move[3])
    tow_column = letter_2_num[move[2]]
    to_output_layer[to_row, tow_column] = 1

    return np.stack([from_output_layer, to_output_layer])

In [ ]:
chess_data_raw = pd.read_csv('/kaggle/input/chess-games/chess_games.csv', usecols = ['AN', 'WhiteElo'])
chess_data = chess_data_raw[chess_data_raw['WhiteElo'] > 2200]
del chess_data_raw
gc.collect
chess_data = chess_data[['AN']]
chess_data = chess_data[~chess_data['AN'].str.contains('{')]
chess_data = chess_data[chess_data['AN'].str.len() > 20]
print(chess_data.shape[0])

In [ ]:
chess_data.head

In [ ]:
class ChessDataset(Dataset):

    def __init__(self, games):
        super(ChessDataset, self).__init__()
        self.games = games

    def __len__(self):
        return 250000

    def __getitem__(self, index):
        game_i = np.random.randint(self.games.shape[0])     #choosing random index
        random_game = chess_data['AN'].values[game_i]       # choose game on index
        moves = create_move_list(random_game)                #creating moves list in that random game
        game_state_i = np.random.randint(len(moves)-1)       #choosing random instance from that game
        next_move = moves[game_state_i]                      #that instance will be our target
        moves = moves[:game_state_i]                          #prevous all moves will be input
        board = chess.Board()                                #making a chess board (alphabet wala)
        for move in moves:                                    #saare moves ka board bnana hai ab
            board.push_san(move)                              #SAN notation me bna diya board ab usko matrix me convert karenge
        x = board_2_rep(board)
        y = move_2_rep(next_move, board)
        if game_state_i % 2 == 1:
            x*=-1

        return x, y                    # x = board before making the move         # y = jo move kiya hai uska previous position and current position

In [ ]:
data_train = ChessDataset(chess_data['AN'])
data_train_loader = DataLoader(data_train, batch_size = 32, shuffle=True, drop_last=True)
print(data_train[0])

In [ ]:
# class module(nn.Module):

#     def __init__(self, hidden_size):
#         super(module, self).__init__()
#         self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(hidden_size)
#         self.bn2 = nn.BatchNorm2d(hidden_size)
#         self.activation1 = nn.SELU()
#         self.activation2 = nn.SELU()

#     def forward(self, x):
#         x_input = torch.clone(x)
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.activation1(x)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x += x_input
#         x = self.activation2(x)
#         return x

In [ ]:
import torch
import torch.nn as nn

class module(nn.Module):
    def __init__(self, hidden_size):
        super(module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)  # Added third CNN layer
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.bn3 = nn.BatchNorm2d(hidden_size)  # BatchNorm for the third layer
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()
        self.activation3 = nn.SELU()  # Activation for the third layer

        # Initialize weights for the third convolution layer to 10^-7
        nn.init.constant_(self.conv1.weight, 1e-7)
        nn.init.constant_(self.conv2.weight, 1e-6)
        nn.init.constant_(self.conv3.weight, 1e-6)

    def forward(self, x):
        x_input = torch.clone(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation2(x)
        x = self.conv3(x)  # Apply the third convolution layer
        x = self.bn3(x)
        x += x_input
        x = self.activation3(x)
        return x


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers

# class Module(layers.Layer):
#     def __init__(self, hidden_size):
#         super(Module, self).__init__()
#         self.conv1 = layers.Conv2D(hidden_size, 3, padding='same')
#         self.conv2 = layers.Conv2D(hidden_size, 3, padding='same')
#         self.bn1 = layers.BatchNormalization()
#         self.bn2 = layers.BatchNormalization()
#         self.activation1 = layers.Activation('selu')
#         self.activation2 = layers.Activation('selu')

#     def call(self, inputs):
#         x_input = tf.identity(inputs)
#         x = self.conv1(inputs)
#         x = self.bn1(x)
#         x = self.activation1(x)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x += x_input
#         x = self.activation2(x)
#         return x


In [ ]:
# class ChessNet(tf.keras.Model):
#     def __init__(self, hidden_layers=4, hidden_size=200):
#         super(ChessNet, self).__init__()
#         self.hidden_layers = hidden_layers
#         self.input_layer = layers.Conv2D(hidden_size, 3, padding='same')
#         self.module_list = [Module(hidden_size) for _ in range(hidden_layers)]
#         self.output_layer = layers.Conv2D(2, 3, padding='same')

#     def call(self, inputs):
#         x = self.input_layer(inputs)
#         x = tf.nn.relu(x)

#         for i in range(self.hidden_layers):
#             x = self.module_listi

#         x = self.output_layer(x)

#         return x


In [ ]:
class ChessNet(nn.Module):

    def __init__(self, hidden_layers=4, hidden_size=200):
        super(ChessNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):

        x = self.input_layer(x)
        x = F.relu(x)

        for i in range(self.hidden_layers):
            x = self.module_list[i](x)

        x = self.output_layer(x)

        return x

In [ ]:
def checkmate_single(board):
    board = board.copy()
    legal_moves = list(board.legal_moves)
    for move in legal_moves:
        board.push_uci(str(move))
        if board.is_checkmate():
            move = board.pop()
            return move
        _ = board.pop()
    return None

In [ ]:
#if pawn diagonally behind a piece(queen,rook,bishop). kill him

In [ ]:
def distribution_over_moves(vals):
    probs = np.array(vals)
    probs = np.exp(probs)
    probs = probs / probs.sum()
    probs = probs ** 3
    probs = probs / probs.sum()
    return probs

In [ ]:
def predict(x):
    model.eval()
    with torch.no_grad():
        outputs = model(x)
        return outputs.cpu().numpy()

In [ ]:
def choose_move(board,color):

    legal_moves = list(board.legal_moves)

    move = checkmate_single(board)   #checking if single move me possible hai check mate

    if move is not None:   #if yes..kardo
        return move

    x = torch.Tensor(board_2_rep(board)).float().to('cuda')
    if color == chess.BLACK:
        x *= -1
    x = x.unsqueeze(0)
    move = predict(x)
    # print(move)
    vals = []
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))
    for from_ in froms:
        # print(move[0,:,:][0][0])
        val = move[0,:,:][0][8-int(from_[1]), letter_2_num[from_[0]]]
        # print(from_)
        vals.append(val)

    probs = distribution_over_moves(vals)

    chosen_from = str(np.random.choice(froms, size=1, p=probs)[0])[:2]

    vals = []
    for legal_move in legal_moves:
        from_ = str(legal_move)[:2]
        if from_ == chosen_from:
            to = str(legal_move)[2:]
            # print(move[0,:,:][0])
            # print(move[0,:,:][1])
            val = move[0,:,:][1][8 - int(to[1]), letter_2_num[to[0]]]
            vals.append(val)
        else:
            vals.append(0)
    chosen_move = legal_moves[np.argmax(vals)]
    return chosen_move

In [ ]:
metric_from = nn.CrossEntropyLoss()
metric_to = nn.CrossEntropyLoss()

In [ ]:
torch.cuda.is_available()           #checking if gpu is available

In [ ]:
import torch.optim as optim

model = ChessNet(hidden_layers=4, hidden_size=200).to('cuda')
optimizer = optim.Adam(model.parameters(), lr=0.0003)
record = []
num_epochs = 5
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(data_train_loader):
        inputs = inputs.float().to('cuda')
        labels = labels.float().to('cuda')  # convert labels to float and using gpu
        optimizer.zero_grad()
        outputs = model(inputs)
        output_from = outputs[:, 0, :]
        output_to = outputs[:, 1, :]
        y_from = labels[:, 0, :]
        y_to = labels[:, 1, :]
        loss_from = nn.CrossEntropyLoss()(output_from, y_from.argmax(dim=1))   #computing loss of choosing move
        loss_to = nn.CrossEntropyLoss()(output_to, y_to.argmax(dim=1))          #loss for chossing next position
        loss = loss_from + loss_to
        loss.backward()                                       # back propagating
        optimizer.step()
        record.append(loss.item())
        if i % 1000 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(data_train_loader), loss.item()))
    torch.save(model.state_dict(), 'model2.pth')

Epoch [1/5], Step [1/7812], Loss: 4.1452
Epoch [1/5], Step [1001/7812], Loss: 0.8552
Epoch [1/5], Step [2001/7812], Loss: 0.7579
Epoch [1/5], Step [3001/7812], Loss: 0.7615
Epoch [1/5], Step [4001/7812], Loss: 0.7103
Epoch [1/5], Step [5001/7812], Loss: 0.6642
Epoch [1/5], Step [6001/7812], Loss: 0.7064
Epoch [1/5], Step [7001/7812], Loss: 0.7537
Epoch [2/5], Step [1/7812], Loss: 0.6845
Epoch [2/5], Step [1001/7812], Loss: 0.6190
Epoch [2/5], Step [2001/7812], Loss: 0.6048
Epoch [2/5], Step [3001/7812], Loss: 0.5640
Epoch [2/5], Step [4001/7812], Loss: 0.6555
Epoch [2/5], Step [5001/7812], Loss: 0.5574
Epoch [2/5], Step [6001/7812], Loss: 0.6992
Epoch [2/5], Step [7001/7812], Loss: 0.7202
Epoch [3/5], Step [1/7812], Loss: 0.6262
Epoch [3/5], Step [1001/7812], Loss: 0.6740
Epoch [3/5], Step [2001/7812], Loss: 0.6623
Epoch [3/5], Step [3001/7812], Loss: 0.6519
Epoch [3/5], Step [4001/7812], Loss: 0.7012
Epoch [3/5], Step [5001/7812], Loss: 0.5893
Epoch [3/5], Step [6001/7812], Loss: 0.58

In [ ]:
board = chess.Board()  #i can give fen string as input to make chess bot
board

In [ ]:
legal = choose_move(board,chess.BLACK)
print("bot moves :: ")
print(legal)
# If there are any legal moves, make the first one
board.push(legal)
board

In [ ]:
move = chess.Move.from_uci("g7g5")

if move in board.legal_moves:
    board.push(move)
else:
    print("Move is not legal")
board